# Importing necessary libraries

In [89]:
# uses for data analysis 
import pandas as pd
#uses neural networks to create word embeddings
import gensim
from gensim.models import Word2Vec, KeyedVectors

# using for tokenization
import nltk

In [90]:
df = pd.read_csv('/Users/vamshikrishna/Downloads/reddit_worldnews_start_to_2016-11-22.csv')

In [91]:
df

,time_created,date_created,up_votes,down_votes,title,over_18,author,subreddit
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes,False,polar,worldnews
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border,False,polar,worldnews
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all,False,fadi420,worldnews
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist,False,mhermans,worldnews
...,...,...,...,...,...,...,...,...
509231,1479816764,2016-11-22,5,0,Heil Trump : Donald Trump s alt-right white...,False,nonamenoglory,worldnews
509232,1479816772,2016-11-22,1,0,There are people speculating that this could b...,False,SummerRay,worldnews
509233,1479817056,2016-11-22,1,0,Professor receives Arab Researchers Award,False,AUSharjah,worldnews
509234,1479817157,2016-11-22,1,0,Nigel Farage attacks response to Trump ambassa...,False,smilyflower,worldnews


# Preprocessing 

In [92]:
prepro_text = df.title.apply(gensim.utils.simple_preprocess)

In [93]:
prepro_text

0                   [scores, killed, in, pakistan, clashes]
1                     [japan, resumes, refuelling, mission]
2                    [us, presses, egypt, on, gaza, border]
3         [jump, start, economy, give, health, care, to,...
4         [council, of, europe, bashes, eu, un, terror, ...
                                ...                        
509231    [heil, trump, donald, trump, alt, right, white...
509232    [there, are, people, speculating, that, this, ...
509233      [professor, receives, arab, researchers, award]
509234    [nigel, farage, attacks, response, to, trump, ...
509235    [palestinian, wielding, knife, shot, dead, in,...
Name: title, Length: 509236, dtype: object

In [94]:
prepro_text.loc[0]

['scores', 'killed', 'in', 'pakistan', 'clashes']

# Model 

In [27]:
model = Word2Vec(
    window=6,
    min_count=1,
    workers=5,
)

#### Build vocabulary from a sequence of sentences

In [28]:
model.build_vocab(prepro_text, progress_per=1000)

##### traing the model

In [30]:
model.train(prepro_text, total_examples=model.corpus_count, epochs=6)

(36603221, 42137136)

In [33]:
model.save("/Users/vamshikrishna/Downloads/word2vec.model")

In [86]:
model.wv.most_similar("killed")

[('kills', 0.807667076587677),
 ('dead', 0.7548298835754395),
 ('wounded', 0.7540781497955322),
 ('injured', 0.6901158094406128),
 ('died', 0.6847264170646667),
 ('kill', 0.6815840005874634),
 ('murdered', 0.6700649261474609),
 ('killing', 0.6633309721946716),
 ('massacred', 0.6453139185905457),
 ('executed', 0.642566978931427)]

In [115]:
len(model.wv['executed'])

100

In [111]:
vocab_len

83265

#### Given a list of words (vocabulary), write a function that will take a word as an argument and return words from the given list that are similar to the given word.

#### preparing a list of all words(lists_of_all_texts) from the preprocessed text 

In [98]:
lists_of_lists_of_all_texts = [] 
for lis in prepro_text:
    lists_of_lists_of_all_texts.append(lis)

In [99]:
len(lists_of_lists_of_all_texts)

509236

#### making a make a flat list out of a list of lists

In [274]:
flat_list = [item for sublist in lists_of_lists_of_all_texts for item in sublist]

# Function for similar words

In [376]:
# list of the unique words 
vectors_word = list(set(flat_list))

In [377]:
# taking the vector of every word
vec2 = []
for words in vectors_word:
    vec2.append(model.wv[words])

In [378]:
# Making a dictionary with words and vectors 
vecdic = {vectors_word[i]: vec2[i] for i in range(len(vectors_word))}

In [379]:
len(vecdic)

83265

# Cosine similarity 
- Cos theta = dotproduct of vectors/cross product of the two vectors
- Costheta range (1, -1)
- theta increases similarity decreases( cos 90 = 0 , cos 0 = 1)
- cos theta = 0 means distance between the vectors is very less(similar vectors) 
- cos theta = 1 means distance between the vectors is very high (not similar)
- Cosin distnace = 1 - cosine similarity 

In [380]:
# Cosine formula 
import math
def cosine_measure(v1, v2):
    prod = np.dot(v1, v2)
    len1 = math.sqrt(np.dot(v1, v1))
    len2 = math.sqrt(np.dot(v2, v2))
    cosinesim = prod / (len1 * len2)
    return cosinesim, v2

##### function to identify the key(word) from the value(vector)

In [381]:
def get_key(val):
    for key, value in vecdic.items():
        if (val == value).all():
                return key

#### Function finding the similar words
- The "cosine_measure" function computes the cosinesimilarity value between two vectors and returns both the vector and the cosinesimilarity value.
- The "get_key" function returns words of the vectors
- If any two vectors have a similarity value greater than 0.6, we consider them to be similar words.

In [395]:
def find_similar_words(word):
    simiar_words = []
    v1 = model.wv[word]
    for v2 in vec2:
        cosinsim_value, vector = cosine_measure(v1, v2)
        if cosinsim_value > 0.6:
            word_name = get_key(vector)
            simiar_words.append((word_name, cosinsim_value))
    return simiar_words

In [396]:
similar_words = find_similar_words("killed")

In [397]:
similar_words 

[('injured', 0.6901157417418422),
 ('wounded', 0.7540781059159559),
 ('kill', 0.6815839925007499),
 ('murdered', 0.6700649143568282),
 ('die', 0.6063821183465053),
 ('beheaded', 0.6219480484216918),
 ('slain', 0.6082477623574142),
 ('slaughtered', 0.6114281086926594),
 ('dead', 0.7548300161047752),
 ('assassinated', 0.6056148884391435),
 ('massacred', 0.6453139654098518),
 ('kills', 0.8076671559793255),
 ('died', 0.6847264181622302),
 ('killing', 0.663331027240474),
 ('executed', 0.642566947812221),
 ('kidnapped', 0.6067576141076226),
 ('killed', 1.0)]

In [398]:
len(vectors_word)

83265

In [399]:
len(set(flat_list))

83265

# function that will take a word as an argument and return words from the given list that are similar to the given word

In [400]:
def synomyms(word):
    synonyms = []
    synss = []
        # Finding the synonyms of the words 
    for mixsyn in find_similar_words(word):
        for syns in mixsyn[:1]:
            synonyms.append(syns) 
                # saving the synonyms of the words as list 
    # taking the words from the synonyms list            
    for similar_syn in set(synonyms):       
        # if the synonym in the "text" it will copy that words to synss = [] list
        if similar_syn in set(flat_list):
            synss.append(similar_syn)
    return synss
x = synomyms("killed")

In [401]:
x

['massacred',
 'kill',
 'died',
 'slaughtered',
 'assassinated',
 'executed',
 'kidnapped',
 'injured',
 'beheaded',
 'kills',
 'dead',
 'killing',
 'die',
 'wounded',
 'murdered',
 'slain',
 'killed']